In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [ ]:
# parameter settings
is_plot = True
start=0
stride = 12
end=120

"""
landmarks_path = '../data/landmarks/6kmh_softbra_8markers_12fps.pkl'
meshes_path = '../data/meshes/6kmh_softbra_8markers/'
test_landmarks_path = '../data/test/softbra_random_landmarks.pkl'
"""

landmarks_path = '../data/landmarks/6kmh_braless_18markers_12fps.pkl'
meshes_path = '../data/meshes/6kmh_braless_26markers/'
test_landmarks_path = '../data/test/braless_random_landmarks.pkl'

# Data Loading

Load landmarks data

_P.S. Sine the construction of RBF motion model requires access to some landmarks data, it's split into the training and testing dataset_

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object(landmarks_path)
landmarks.interp_field()

In [ ]:
landmarks_test, landmarks_train = landmarks.split(('marker 5', ))

Load meshes data

In [ ]:
from mesh4d import obj3d
from mesh4d.analysis import reg_rbf

o3_ls = obj3d.load_obj_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
    obj_type=reg_rbf.Obj3d_RBF,
)

# Displacement Field Estimation

In [ ]:
o4 = reg_rbf.Obj4d_RBF(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=True,
)
o4.add_obj(*o3_ls, landmarks=landmarks_train)
o4.load_markerset('landmarks_train', landmarks_train)
o4.load_markerset('landmarks_test', landmarks_test)

In [ ]:
import pyvista as pv
pv.set_jupyter_backend('static')

if is_plot:
    o3_ls[0].show()

In [ ]:
if is_plot:
    o3_ls[0].show_deform(mode='nonrigid')

In [ ]:
if is_plot:
    o3_ls[0].show_deform(mode='rigid')

In [ ]:
obj_deform = o3_ls[0].get_deform_obj3d()
obj_gt = o3_ls[1]

if is_plot:
    obj3d.Obj3d_Kps.show_diff(obj_deform, obj_gt)

# Ultra-dense Motion Capture

In [ ]:
kps_source = landmarks_test.get_time_coord(0)
o4.vkps_track(kps_source, frame_id=0)

In [ ]:
if is_plot:
    o4.show_deform_gif(output_folder='output', filename='obj4d_vkps_rbf', kps_names=('vkps', 'landmarks_test'))

Assemble the tracked virtual key points

In [ ]:
vkps = o4.assemble_markerset(name='vkps')

In [ ]:
if is_plot:
    vkps.plot_frame(frame_id=0)

In [ ]:
from mesh4d import kps
diff = kps.MarkerSet.diff(vkps, landmarks_test)

In [ ]:
utils.save_pkl_object(diff, 'output/diff_rbf.pkl')

# Randomly Selected Landmarks

In [ ]:
from mesh4d import utils

test_landmarks = utils.load_pkl_object(test_landmarks_path)
test_kps = test_landmarks.get_frame_coord(0)

In [ ]:
o4.vkps_track(test_kps, frame_id=0, name='test')

In [ ]:
if is_plot:
    o4.show_deform_gif(output_folder='output', filename='obj4d_vkps_rbf_random', kps_names=('test',))